# Capture patterns with recurrent neural networks
## Recurrent neural networks
### In the previous module, we have been using rich semantic representations of text, and a simple linear classifier on top of the embeddings. What this architecture does is to capture aggregated meaning of words in a sentence, but it does not take into account the order of words, because aggregation operation on top of embeddings removed this information from the original text. Because these models are unable to model word ordering, they cannot solve more complex or ambiguous tasks such as text generation or question answering.

### To capture the meaning of text sequence, we need to use another neural network architecture, which is called a recurrent neural network, or RNN. In RNN, we pass our sentence through the network one word vector from a news article sequence at a time, and the network produces some state, which we then pass to the network again with the next one word vector from the sequence. RNN storing a "memory" of the previous in the state, helps the network understand the context of the sentence to be able to predict the network word in the sequence.

In [2]:
!pip install torchinfo

In [ ]:
import torch
import torchtext
from torchinfo import summary
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)

## Simple RNN classifier
### In the case of simple RNN, each recurrent unit is a simple linear network, which takes concatenated input vector and state vector, and produce a new state vector. PyTorch represents this unit with RNNCell class, and a networks of each cells - as RNN layer.

### To define an RNN classifier, we will first apply an embedding layer to lower the dimensionality of input vocabulary, and then have a RNN layer on top of it:

In [4]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

### In our case, we will use padded data loader, so each batch will have a number of padded sequences of the same length. RNN layer will take the sequence of embedding tensors, and produce two outputs:

### The input to the embedding layer is the word sequence or news article
### The embedding layer output contains the vector index value in vocab for each word in the sequence
### x is a sequence of RNN cell outputs at each step.
###  h is a final hidden state for the last element of the sequence. Each RNN hidden layer stores the prior word in the sequence and the current as each word in the sequence is passed through the layers.
### We then apply a fully-connected linear classifier to get the probability for number of classes.

### Note: RNNs are quite difficult to train, because once the RNN cells are unrolled along the sequence length, the resulting number of layers involved in back propagation is quite large. Thus we need to select small learning rate, and train the network on larger dataset to produce good results. It can take quite a long time, so using GPU is preferred.

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

### Now, let's load the test dataset to evaluate the trained RNN model. We'll be using the 4 different classes of the news categories to map the predicted output with the targeted label.

In [ ]:
print(f'class map: {classes}')

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### Before we evaluate the model, we'll extract the padded vector dataset from the dataloader. We will use the vocab.itos function to convert the numeric index to the word it matches in the vocabulary. When conversion from numeric to string happens for padded vectors, the '0' values are set to a special character 'unk'  as an unknown identifier. So, the character needs to be removed, depending on the unknown values from the padded zeros.

### Finally, we’ll run the model with our test dataset to verify if the expected output matched the predicted.

In [ ]:
net.eval()

with torch.no_grad():
    for batch_idx, (target, data) in enumerate(test_loader):
        
        word_lookup = [vocab.itos[w] for w in data[batch_idx]]
        unknow_vals = {'<unk>'}
        word_lookup = [ele for ele in word_lookup if ele not in unknow_vals]
        print('Input text:\n {}\n'.format(word_lookup))
        
        data, target = data.to(device), target.to(device)
        pred = net(data)
        print(torch.argmax(pred[batch_idx]))
        print("Actual:\nvalue={}, class_name= {}\n".format(target[batch_idx], classes[target[batch_idx]]))
        print("Predicted:\nvalue={}, class_name= {}\n".format(pred[0].argmax(0),classes[pred[0].argmax(0)]))
        break

## Long Short Term Memory (LSTM)
### One of the main problems of classical RNNs is the so-called vanishing gradients problem. Because RNNs are trained end-to-end in one back-propagation pass, it is having hard times propagating error to the first layers of the network, and thus the network cannot learn relationships between distant tokens. The gradient helps in adjusting the weights during back-progagation to achieve better accuracy and reduce the error margin. If the weights are too small the network does not learn. Since the gradient decreases during back-propagation in RNNs, the network does not learn the initial inputs in the network. In other ways, the network "forgets" the earlier word inputs.

### One of the ways to avoid this problem is to introduce explicit state management by using so called gates. There are two most known architectures of this kind: Long Short Term Memory (LSTM) and Gated Relay Unit (GRU).

###  While internal structure of LSTM cell may look complex, PyTorch hides this implementation inside the LSTMCell class, and provides a LSTM object to represent the whole LSTM layer. Thus, implementation of LSTM classifier will be pretty similar to the simple RNN which we have seen above:

In [ ]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,(h,c) = self.rnn(x)
        return self.fc(h[-1])

### Now let's train our network. Note that training LSTM is also quite slow, and you may not seem much raise in accuracy in the beginning of training. Also, you may need to play with lr learning rate parameter to find the learning rate that results in reasonable training speed.

In [ ]:
net = LSTMClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

## Packed sequences
### In our example, we had to pad all sequences in the minibatch with zero vectors. While it results in some memory waste, with RNNs it is more critical that additional RNN cells are created for the padded input items, which take part in training, yet do not carry any important input information. It would be much better to train RNN only to the actual sequence size.

### To do that, a special format of padded sequence storage is introduced in PyTorch. Suppose we have input padded minibatch which looks like this:

### [[1,2,3,4,5],
### [6,7,8,0,0],
### [9,0,0,0,0]]


### Here 0 represents padded values, and the actual length vector of input sequences is [5,3,1].

### In order to effectively train RNN with padded sequence, we want to begin training first group of RNN cells with large minibatch ([1,6,9]), but then end processing of third sequence, and continue training with shorted minibatches ([2,7], [3,8]), and so on. Thus, packed sequence is represented as one vector - in our case [1,6,9,2,7,3,8,4,5], and length vector ([5,3,1]), from which we can easily reconstruct the original padded minibatch.

### To produce packed sequence, we can use torch.nn.utils.rnn.pack_padded_sequence function. All recurrent layers, including RNN, LSTM and GRU, support packed sequences as input, and produce packed output, which can be decoded using torch.nn.utils.rnn.pad_packed_sequence.

### To be able to produce packed sequence, we need to pass length vector to the network, and thus we need a different function to prepare minibatches:

In [ ]:
def pad_length(b):
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v]),
        torch.tensor(len_seq)
    )

train_loader_len = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)
test_loader_len = torch.utils.data.DataLoader(test_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)

### The actual network would be very similar to LSTMClassifier above, but forward pass will receive both padded minibatch and the vector of sequence lengths. After computing the embedding, we compute packed sequence, pass it to LSTM layer, and then unpack the result back.



### Note: We actually do not use unpacked result x, because we use output from the hidden layers in the following computations. Thus, we can remove the unpacking altogether from this code. The reason we place it here is for you to be able to modify this code easily, in case you should need to use network output in further computations.


In [6]:
class LSTMPackClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        _,(h,c) = self.rnn(pad_x)
        return self.fc(h[-1])

### Now let's train our network with the padded sequence:

In [ ]:
net = LSTMPackClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net,train_loader_len, lr=0.001,use_pack_sequence=True)


### Note: You may have noticed the parameter use_pack_sequence that we pass to the training function. Currently, pack_padded_sequence function requires length sequence tensor to be on CPU device, and thus training function needs to avoid moving the length sequence data to GPU when training. You can look into implementation of train_epoch_emb helper function in the torchnlp.py file located in the local directory.

In [ ]:
net.eval()

with torch.no_grad():
    for label,text,off in test_loader_len:
        
        text, label = text.to(device), label.to(device)
        off = off.to('cpu')
        print(f'off value: {off}')
        pred = net(text, off )
        print(f'target {label}')
        y=torch.argmax(pred, dim=1)
        print(f'pred: {y}')
        print("Predicted:\nvalue={}, class_name= {}\n".format(y[0],classes[y[0]]))
        print("Target:\nvalue={}, class_name= {}\n".format(label[0],classes[label[0]]))
        break
     

## Bidirectional and multilayer RNNs
### In our examples, all recurrent networks operated in one direction, from beginning of a sequence to the end. It looks natural, because it resembles the way we read and listen to speech. However, since in many practical cases we have random access to the input sequence, it might make sense to run recurrent computation in both directions. Such networks are call bidirectional RNNs, and they can be created by passing bidirectional=True parameter to RNN/LSTM/GRU constructor.

### Example: self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True, bidrectional=True)


### When dealing with bidirectional network, we would need two hidden state vectors, one for each direction. PyTorch encodes those vectors as one vector of twice larger size, which is quite convenient, because you would normally pass the resulting hidden state to fully-connected linear layer, and you would just need to take this increase in size into account when creating the layer.

### Recurrent network, one-directional or bidirectional, captures certain patterns within a sequence, and can store them into state vector or pass into output. As with convolutional networks, we can build another recurrent layer on top of the first one to capture higher level patterns, build from low-level patterns extracted by the first layer. This leads us to the notion of multi-layer RNN, which consists of two or more recurrent networks, where output of the previous layer is passed to the next layer as input.

### PyTorch makes constructing such networks an easy task, because you just need to pass num_layers parameter to RNN/LSTM/GRU constructor to build several layers of recurrence automatically. This would also mean that the size of hidden/state vector would increase proportionally, and you would need to take this into account when handling the output of recurrent layers.